<a href="https://colab.research.google.com/github/Sean1572/SoupDumplings/blob/Sean1572-test-code/Experimental_Mode_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip3 install segmentation_models_pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from  torchvision.models.segmentation import  deeplabv3_resnet50
import os
import segmentation_models_pytorch as smp
import os
from numpy import asarray
import numpy as np
from PIL import Image
#/content/drive/Shareddrives/ACM_Project_Team_3 


# Raw images
#/content/drive/Shareddrives/ACM_Project_Team_3/HYTA-master/images/training

# Labels 
# /content/drive/Shareddrives/ACM_Project_Team_3/HYTA-master/3GT/training


from torch.utils.data import Dataset
from natsort import natsorted
class CustomDataSet(Dataset):
    def __init__(self, main_dir, label_dir, transform):
        self.main_dir = main_dir
        self.label_dir = label_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        all_segs = os.listdir(main_dir)
        self.total_imgs = natsorted(all_imgs)
        self.total_segs = natsorted(all_segs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        seg_loc = os.path.join(self.label_dir, self.total_segs[idx])
        labeled_image = Image.open(seg_loc).convert("RGB")
        transform = transforms.Compose([transforms.ToTensor()])
        labeled_image = transform(labeled_image)
        labeled_image = labeled_image.float()
        return tensor_image, labeled_image

BATCH_SIZE = 32

## transformations
transform = transforms.Compose([transforms.Resize((450, 480)), transforms.ToTensor()])

rgb_img = '/content/drive/Shareddrives/ACM_Project_Team_3/WSISEG-Database-master/whole sky images'
seg_img = '/content/drive/Shareddrives/ACM_Project_Team_3/WSISEG-Database-master/annotation'
## download and load training dataset
imagenet_data = CustomDataSet(rgb_img, seg_img, transform=transform)
trainloader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=2)

rgb_img = '/content/drive/Shareddrives/ACM_Project_Team_3/HYTA-master/3GT/training'
seg_img = '/content/drive/Shareddrives/ACM_Project_Team_3/HYTA-master/images/training'
## download and load training dataset
imagenet_data = CustomDataSet(rgb_img, seg_img, transform=transform)
testloader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=2)

#rgb_img_t = '/content/drive/MyDrive/Highway_Dataset/Test/TestSeq04/image'
#seg_img_t = '/content/drive/MyDrive/Highway_Dataset/Test/TestSeq04/label'
### download and load training dataset
#imagenet_data_test = CustomDataSet(rgb_img_t, seg_img_t, transform=transform)
#testloader = torch.utils.data.DataLoader(imagenet_data_test,
#                                          batch_size=BATCH_SIZE,
#                                          shuffle=False,
#                                          num_workers=2)
#

In [63]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)
        self.upsample = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop1 = nn.Dropout(0.2)
        self.pool1 = nn.MaxPool2d((2,2))

    def layerDown(self, x):
        x = self.conv1(x)
        x = self.drop1(x)
        x = F.relu(x)
        return x
    
    #yet to test this layer design 
    def layerUpTest(self, x):
        x = self.conv1(x)
        x = self.drop1(x)
        x = self.convUp1(x)
        x = F.relu(x)
        return x

    def layerUp(self, x):
        x = self.convUp1(x)
        x = self.drop1(x)
        x = F.relu(x)
        return x
    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        #x = self.layerDown(x)
        #x = self.layerDown(x)
        x = self.layerDown(x)
        x = self.layerDown(x)
        x = self.layerDown(x)
        #x = self.layerUp(x)
        #x = self.layerUp(x)
        x = self.layerUp(x)
        x = self.layerUp(x)
        x = self.layerUp(x)
   

        # flatten => 32 x (32*26*26)
        #x = x.flatten(start_dim = 1)

        #32 x (32*26*26) => 32x128
        #x = self.d1(x)
        #x = F.relu(x)
        #x = x.reshape([32, 3, 28, 28])
        #x = self.d2(x)
        #x = F.relu(x)

        # logits => 32x10
        logits = x
        out = F.softmax(logits, dim=1)
        return out

  ## test the model with 1 batch
model = MyModel()
for images, labels in trainloader:
    print("batch size:", images.shape)
    out = model(images)
    print(out.shape, labels.shape)
    break


batch size: torch.Size([32, 3, 450, 480])
torch.Size([32, 3, 450, 480]) torch.Size([32, 3, 450, 480])


In [66]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)
        self.convUp1 = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=3)
        self.upSampling = nn.Upsample(in_channels=3, out_channels=3, kernel_size=3)
        self.drop1 = nn.Dropout(0.2)
        self.pool1 = nn.MaxPool2d((2,2))

    def layerDown(self, x):
        x = self.conv1(x)
        x = self.drop1(x)
        x = F.relu(x)
        return x
    
    #yet to test this layer design 
    def layerUpTest(self, x):
        x = self.conv1(x)
        x = self.drop1(x)
        x = self.convUp1(x)
        x = F.relu(x)
        return x

    def layerUp(self, x):
        x = self.convUp1(x)
        x = self.drop1(x)
        x = F.relu(x)
        return x
    def forwardBeta(self, x):
        # 32x1x28x28 => 32x32x26x26
        #x = self.layerDown(x)
        #x = self.layerDown(x)
        x = self.layerDown(x)
        x = self.layerDown(x)
        x = self.layerDown(x)
        #x = self.layerUp(x)
        #x = self.layerUp(x)
        x = self.layerUp(x)
        x = self.layerUp(x)
        x = self.layerUp(x)
   

        # flatten => 32 x (32*26*26)
        #x = x.flatten(start_dim = 1)

        #32 x (32*26*26) => 32x128
        #x = self.d1(x)
        #x = F.relu(x)
        #x = x.reshape([32, 3, 28, 28])
        #x = self.d2(x)
        #x = F.relu(x)

        # logits => 32x10
        logits = x
        out = F.softmax(logits, dim=1)
        return out


    def forward(self, x):
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.pool1(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.pool1(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.pool1(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.pool1(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.pool1(x)
      x = self.upSampling(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.upSampling(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.upSampling(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.upSampling(x)
      x = self.conv1(x)
      x = self.conv1(x)
      x = self.upSampling(x)
      x = self.conv1(x)
      x = self.conv1(x)

      logits = x
      out = F.softmax(logits, dim=1)
      return out

  ## test the model with 1 batch
model = MyModel()
for images, labels in trainloader:
    print("batch size:", images.shape)
    out = model(images)
    print(out.shape, labels.shape)
    break



TypeError: ignored

In [5]:
import segmentation_models_pytorch as smp
learning_rate = 5
num_epochs = 10

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model = MyModel()
#model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
# model to eval() model and load onto computation devicce
#model.eval().to(device)
#model = model.to(device)
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

DEVICE = 'cpu'

train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)


## Model
#ENCODER = 'se_resnext50_32x4d'
#ENCODER_WEIGHTS = 'imagenet'
#CLASSES = ['car']
#ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
#DEVICE = 'cpu' # did not get 'cuda' doesn't work
#
## create segmentation model with pretrained encoder
#model = smp.FPN(
#    encoder_name=ENCODER, 
#    encoder_weights=ENCODER_WEIGHTS, 
#    classes=len(CLASSES), 
#    activation=ACTIVATION,
#)
#
#preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)



max_score = 0

for i in range(0, 10):

  print('\nEpoch: {}'.format(i))
  train_logs = train_epoch.run(trainloader)
  valid_logs = valid_epoch.run(trainloader)
  
  # do something (save model, change lr, etc.)
  if max_score < valid_logs['iou_score']:
      max_score = valid_logs['iou_score']
      torch.save(model, './best_model.pth')
      print('Model saved!')
      
  if i == 25:
      optimizer.param_groups[0]['lr'] = 1e-5
      print('Decrease decoder learning rate to 1e-5!')
  model.eval()
#print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
#      %(i, 0 / (i+1), train_acc/(i+1)))        


Epoch: 0
train:   0%|          | 0/13 [00:26<?, ?it/s]


RuntimeError: ignored

In [33]:
test_acc = 0.0
for i, (images, labels) in enumerate(trainloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_acc += compute_accuracy(outputs, labels)
    
    valid_logs = valid_epoch.run(testloader)
    max_score = valid_logs['iou_score']
    torch.save(model, './best_model.pth')
    print('iou score ' + str(valid_logs['iou_score']))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b21070c20>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b21070c20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child

valid:   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: ignored